In [ ]:
import os
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, col, explode_outer, struct, from_json
from pyspark.sql.types import StructType, StructField, ArrayType, StringType, LongType, IntegerType
from delta import *
import findspark

findspark.init()

warehouse_diretory_path = '[YOUR_WAREHOUSE]'
ensembl_gene_annotation_file = 'file:///[FILE_FULL_NAME:homo_sapiens.grch38.108.entrez.tsv]'
biological_database_name = 'biological_database'
ensembl_gene_annotation_table_name = 'bronze_ensembl_gene_entrez_annotation'

conf = SparkConf()
conf.setAll(
    [
        ('spark.master', 'local[*]'), 
        ('spark.driver.host', 'localhost'),
        ('spark.app.name', 'Ensembl Gene Entrez Annotation Importer'),
        ('spark.ui.showConsoleProgress', 'true'),
        ('spark.sql.execution.arrow.pyspark.enabled', 'false'),                   
        ('spark.sql.execution.arrow.pyspark.fallback.enabled', 'true'),
        ('spark.dynamicAllocation.enabled', 'false'),
        ('spark.sql.caseSensitive', 'true'),
        ('spark.sql.adaptive.enabled', 'true'),
        ('spark.sql.extensions','io.delta.sql.DeltaSparkSessionExtension'),
        ('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog'),
        ('spark.sql.warehouse.dir', warehouse_diretory_path),
        ('spark.driver.extraJavaOptions', f'-Dderby.system.home={warehouse_diretory_path}')
    ])

spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()

In [ ]:
ensembl_gene_annotation_df = spark.read \
    .option('header', True) \
    .options(delimiter='\t') \
    .csv(ensembl_gene_annotation_file)

In [ ]:
spark.sql(f'CREATE DATABASE IF NOT EXISTS {biological_database_name};')

In [ ]:
spark.sql(f'USE {biological_database_name};')

In [ ]:
ensembl_gene_annotation_df.write \
    .format('delta') \
    .mode('overwrite') \
    .option('overwriteSchema', 'true') \
    .option('partitionOverwriteMode', 'dynamic') \
    .saveAsTable(ensembl_gene_annotation_table_name)

In [ ]:
spark.stop()